In [1]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker, scoped_session, relationship, validates
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, String, Integer, Column, Date, Boolean, sql
from sqlalchemy.schema import ForeignKey, PrimaryKeyConstraint
from sqlalchemy.dialects import postgresql
from functools import partial
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import mozilla_headers
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
import sqlite3

In [2]:
cnn = sqlite3.connect("data")
cur = cnn.cursor()

In [3]:
Column = partial(Column, nullable=False)

In [4]:
def get_connector():
    url = 'postgresql+psycopg2://{}:{}@{}:{}/{}'
    url = url.format("postgres", 
                     "12404888", 
                     "127.0.0.1", "5434", 
                     "headhunter5")
    con = sqlalchemy.create_engine(url, client_encoding='utf8', server_side_cursors=True, echo=False)
    return con

In [5]:
con = get_connector().execution_options(stream_results=True)

session_factory = sessionmaker(bind=con)
Session = scoped_session(session_factory)
# TODO: Might be a good idea to remove in the future and switch to dynamic `Session()` calls entirely while instantiating `DBOps`.
session = Session() # the default one. Not thread-safe.

In [6]:
Base = declarative_base(bind=con)

In [7]:
class Raw(Base):
    __tablename__ = 'raw_html'

    id = Column(Integer, primary_key=True, autoincrement=True)
    html = Column(String)
    when_parsed = Column(Date, server_default=sql.functions.current_date(), onupdate=sql.functions.current_date())
    is_archived = Column(Boolean, nullable=True) # null if failed to fetch
    when_closed = Column(Date, nullable=True) # null if failed to fetch
    http_errcode = Column(Integer, nullable=True) # null if fetch successful

In [8]:
raw = Raw.__table__ # backwards compat

vacancies_skills = Table('vacancies_skills', Base.metadata,
    Column('vacancy_id', Integer, ForeignKey('vacancies.id')),
    Column('skill_id', Integer, ForeignKey('skills.id')),
    PrimaryKeyConstraint('vacancy_id', 'skill_id'),
)

In [9]:
class Vacancy(Base):
    __tablename__ = 'vacancies'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    raw_id = Column(Integer, sqlalchemy.schema.ForeignKey("raw_html.id"))
    open_time = Column(Date)
    text = Column(String)
    salary = Column(Integer)
    skills = relationship(
        "Skill",
        secondary='vacancies_skills',
        back_populates="vacancies")
    
    def __repr__(self):
        return 'Vacancy<{}>'.format(', '.join((
            '{}...'.format(self.text[:50]),
            '{}'.format(self.open_time.isoformat()),
            '{} $'.format(self.salary),
        )))

In [10]:
class Skill(Base):
    __tablename__ = 'skills'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    skill = Column(postgresql.JSONB(none_as_null=True), unique=True)
    vacancies = relationship(
        "Vacancy",
        secondary='vacancies_skills',
        back_populates="skills")
    unique_skill_id = Column(Integer, ForeignKey('unique_skills.id', ondelete="SET NULL"), nullable=True)
    unique_skill = relationship("UniqueSkill", uselist=False, back_populates="skills")
    skill_vector = relationship("SkillVector", uselist=False, back_populates="skill")

    # FIXME: make it impossible to create a skill not associated with a vacancy
    #@validates('vacancies')
    #def validate_vacancies(self, key, vacancies):
    #    assert len(vacancies) > 0
    #    return vacancies
    
    def __repr__(self):
        return 'Skill<{}>'.format(', '.join((
            '{}'.format(' '.join(self.skill)),
            str(self.unique_skill_id),
            'V+' if self.skill_vector else 'V-',
        )))

In [11]:
class SkillVector(Base):
    __tablename__ = 'skillvectors'

    id = Column(Integer, ForeignKey('skills.id'), primary_key=True)
    vector = Column(postgresql.ARRAY(postgresql.DOUBLE_PRECISION))
    skill = relationship("Skill", back_populates="skill_vector")

In [12]:
class UniqueSkill(Base):
    # aka USkill
    __tablename__ = 'unique_skills'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    skills = relationship("Skill", back_populates="unique_skill")
    price = Column(Integer, nullable=True) # null if not calculated yet (price calc happens after merging into uskills)
    
    def __repr__(self):
        return 'UniqueSkill<{}>'.format(',\n'.join(map(str, self.skills)))

In [13]:
def translate(text, dest='ru'):
    translator = Translator(service_urls=['translate.google.com'])
    return translator.translate(text, dest=dest).text

In [14]:
def translate_courses(courses):
    courses = list(map(lambda x: x[0] + ". " + x[1], courses))
    translated_courses = list(map(lambda x: translate(x[:2900]), tqdm_notebook(courses))) + list(map(lambda x: translate(x[:2900], dest="en"), tqdm_notebook(courses)))
    for course in translated_courses:
        cur.execute("insert into courses (title, text, type, url) values ('{0}', '{1}', '{2}', '{3}')"
                    .format(course.split()[0], course, 'translated', '-'))
        cnn.commit()

In [15]:
def translate_tags(tags):
    translate_tags = list(map(lambda x: translate(x), tags))
    for tag in translate_tags:
        cur.execute("insert into tags (text) values ('{0}')".format(tag))
    return translate_tags

In [16]:
def tags_of_vacancy(vacancy, tags):
    tags_of_vacancy = set()
    for j in range(len(tags)):
        tag = tags[j] if tags[j] != "c" and tags[j] != "с" else "&&**^^"
        tag = tag.lower()
        if tag in vacancy:
            tags_of_vacancy.add(tag)
    return tuple(tags_of_vacancy)

In [17]:
def tags_of_course(course, tags):
    tags_of_course = set()
    for j in range(len(tags)):
        tag = tags[j] if tags[j] != "c" and tags[j] != "с" else "&&**^^"
        tag = tag.lower()
        if tag in course:
            tags_of_course.add(tag)
    return list(tags_of_course)

In [18]:
vdata = session.execute(session.query(Vacancy)).fetchall()

In [19]:
ids = list(map(lambda x: x[0], vdata))
texts = list(map(lambda x: x[3], vdata))

In [20]:
tags = list(set(list(map(lambda x: x[0], cur.execute("select * from tags").fetchall()))))

In [21]:
tags.append("C++")
tags = list(set(list(map(lambda x: x.lower(), tags))))

In [22]:
tof = []
for i in tqdm_notebook(range(len(texts))):
    tof.append(tags_of_vacancy(texts[i], tags))

In [23]:
Xy1 = pd.DataFrame()
Xy1['text'] = texts
Xy1['tags'] = list(map(lambda x: x if len(x) else None, tof))
Xy1 = Xy1.dropna()

In [24]:
courses = cur.execute("select * from courses").fetchall()
courses = list(map(lambda x: x[0] + ". " + x[1], courses))

In [25]:
tags_of_courses = [tags_of_course(course, tags) for course in courses]

In [26]:
Xy = pd.DataFrame()
Xy["X"] = courses
Xy["y"] = list(map(lambda x: x if len(x) else None, tags_of_courses))
Xy = Xy.dropna()

In [27]:
Xy = np.array(Xy)
Xy1 = np.array(Xy1)

In [28]:
Xy_courses = np.array(list(map(lambda x: [x[0], set(x[1])], tqdm_notebook(Xy))))
Xy_vacancy = np.array(list(map(lambda x: [x[0], set(x[1])], tqdm_notebook(Xy1))))

In [29]:
Xy_courses.shape, Xy_vacancy.shape

((1359, 2), (43583, 2))

In [30]:
matrix = np.array([np.array([0 for i in range(len(Xy_courses))]) for j in range(len(Xy_vacancy))])

In [31]:
for i in tqdm_notebook(range(len(Xy_vacancy))):
    for j in range(len(Xy_courses)):
        matrix[i][j] = len(Xy_courses[j][1] & Xy_vacancy[i][1])

In [32]:
matrix_norm = np.array([np.array([0 for i in range(len(Xy_courses))]) for j in range(len(Xy_vacancy))])

In [34]:
for i in tqdm_notebook(range(len(Xy_vacancy))):
    for j in range(len(Xy_courses)):
        matrix_norm[i][j] = len(Xy_courses[j][1] & Xy_vacancy[i][1]) / len(Xy_courses[j][1] | Xy_vacancy[i][1])

21153/|/ 49%|| 21153/43583 [01:10<01:14, 302.04it/s]



In [35]:
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

In [36]:
matrix = pd.DataFrame(list(matrix))

In [74]:
matrix = matrix.as_matrix()

In [100]:
U, d, Vt = svds(matrix_norm.astype(float))
d = np.diag(d)

In [113]:
np.dot(np.dot(U, d), Vt)[6000].argmax()

290

In [30]:
vacancy = pd.DataFrame()
vacancy['text'] = texts

In [31]:
c = pd.DataFrame()
courses = cur.execute("select * from courses").fetchall()
c['text'] = [courses[i][0] + ". " + courses[i][1] for i in range(len(courses))]
c['type'] = [courses[i][2] for i in range(len(courses))]
c['url'] = [courses[i][3] for i in range(len(courses))]
courses = c

In [32]:
vacancy.shape, courses.shape

((841447, 1), (2381, 3))

### New tags

In [30]:
import gensim
import pymorphy2

Using TensorFlow backend.


In [31]:
import string
from googletrans import Translator
translator = Translator(service_urls=['translate.google.com'])

In [32]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
morph = pymorphy2.MorphAnalyzer()

In [33]:
def avg_feature_vector(words, num_features):
    global w2v
    model = w2v
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in words:
        nwords = nwords + 1
        try:
            featureVec = np.add(featureVec, model[word])
        except KeyError:
            featureVec = np.add(featureVec, np.zeros(num_features))

    if (nwords > 0):
        featureVec = np.divide(featureVec, nwords)
    return featureVec

In [34]:
def normalize_form(txt):
    global morph
    p = [morph.parse(x)[0].normal_form for x in txt]
    return p

In [35]:
def make_dict(x):
    dict_rus = ' '.join(x)
    translator = str.maketrans({key: ' ' for key in string.punctuation.replace('/', '').replace('-', '') + string.digits + '№'})
    dict_rus = dict_rus.translate(translator).lower()
    dict_rus_split = dict_rus.split()
    return dict_rus_split

In [36]:
def words2vecs(arr):
    arr = list(map(lambda x: x.split(), arr))
    arr = [normalize_form(make_dict(new)) for new in arr]
    arr = [avg_feature_vector(sent, 300) for sent in list(arr)]
    return np.array(arr)

In [37]:
def translate(text, dest='ru'):
    global translator
    return translator.translate(text, dest=dest).text

In [38]:
nvec = words2vecs(['requirements'])[0]

In [39]:
vacancy['text'] = list(map(lambda x: x.lower().replace('\n', ' '), tqdm_notebook(vacancy['text'])))

NameError: name 'vacancy' is not defined

In [ ]:
len(vacancy['text'][883])

In [ ]:
texts = vacancy['text']

In [ ]:
cnt = 0
for i in tqdm_notebook(range(len(texts))):
    if i % 100 == 0:
        print(i + 1 - cnt)
    try:
        texts[i] = translate(texts[i], dest='en')
    except:
        translator = Translator()
        cnt += 1

1
98
198
296
395
494
593
692
791
886
985
1084
1180
1279
1379
1479
1578
1677
1776
1876
1974
2073
2172
2272
2371
2471
2570
2667
2765
2862
2962
3062
3160
3259
3356
3455
3554
3654
3753
3853
3953
4053
4151
4250
4349
4449
4549
4646
4746
4846
4946
5045
5142
5241
5341
5441
5540
5638
5737
5837
5936
6035
6134
6233
6333
6433
6533
6633
6733
6832
6925
7025
7125
7225
7325
7423
7521
7620
7719
7819
7917
8016
8114
8214
8314
8414
8514
8613
8713
8813
8912
9012
9112
9212
9311
9411
9510
9610
9709
9809
9908
10008
10108
10208
10308
10408
10508
10605
10704
10804
10903
11001
11097
11185
11280
11377
11476
11574
11672
11770
11870
11970
12069
12167
12266
12366
12466
12566
12665
12763
12861
12959
13055
13155
14152
14251
14351
14450
14550
14649
14748
14848
14948
15048
15148
15246
15344
15444
15543
15643
15743
15841
15941
16040
16137
16236
16335
16434
16534
16633
16731
16831
16929
17026
17125
17220
17317
17417
17517
17617
17716
17816
17915
18014
18113
18212
18310
18409
18507
18603
18699
18799
18899
18998
19095
19193

In [ ]:
vacancy['text'] = list(map(lambda x: x.split(), vacancy['text']))

In [45]:
new_tags = []
eps = 1e-20

In [49]:
for text in tqdm_notebook(vacancy['text'][10:20]):
    for i in range(len(text)):
        veci = words2vecs([text[i]])[0]
        if np.sum(veci * nvec) / (len(veci) * len(nvec)) <= eps:
            print(text[max(i - 5, 0): i - 1] + text[i + 1:i + 5])
            new_tags.extend(text[max(i - 5, 0): i] + text[i:i + 5])

['a', 'leading', 'western', 'fmcg', 'company', 'with', 'manufacturing', 'in', 'russia', 'is', 'offering', 'excellent', 'opportunity', 'for', 'engineering', 'and', 'project', 'manager', 'the', 'right', 'candidate', 'will', 'have', '·', 'university', 'degree', 'in', 'mechanical,', 'electrical,', 'industrial,', 'chemical,', 'civil', 'engineering', '·', 'international', 'experience', 'in', 'project', 'realization', 'and', 'management', '.', '·', 'min', '8', 'years', 'experience', 'in', 'food', 'or', 'beverage', 'industry.', '·', 'knowledge', 'and', 'experience', 'in', 'manufacturing', 'and/or', 'engineering', 'management', 'in', 'multiple', 'plant', 'environment.', '·', 'responsibility', 'for', 'and', 'management', 'of', 'major', 'capital', 'projects,', 'including', 'civil', 'works.', '·', 'experience', 'and', 'expertise', 'in', 'the', 'areas', 'of', 'sensitive', 'products', 'and/or', 'mineral', 'water', 'are', 'desirable.', '·', 'fluent', 'business', 'english', '(oral', 'and', 'in', 'writ

limit_output extension: Maximum message size of 10000 exceeded with 38215 characters

In [41]:
tags.sort()

In [98]:
tags = set(tags[:len(tags)//2 + 8])

TypeError: 'set' object is not subscriptable

In [99]:
tags = list(tags)

In [100]:
from sklearn.cluster import Birch

In [151]:
n_clusters = 9

In [152]:
brc = Birch(n_clusters=n_clusters)

In [153]:
X = list(map(lambda x: words2vecs([x])[0], tags))

In [154]:
brc.fit(X)

Birch(branching_factor=50, compute_labels=True, copy=True, n_clusters=9,
   threshold=0.5)

In [155]:
import matplotlib
import matplotlib.pyplot as plt

In [156]:
%matplotlib inline

In [157]:
p = brc.fit_predict(X)

In [158]:
clusters = [[] for i in range(n_clusters)]

In [159]:
for i in range(len(tags)):
    clusters[p[i] - 1].append(tags[i])

In [160]:
clusters

[['php', 'html', 'sql'],
 ['java', 'ruby', 'pascal', 'css', 'ios', 'android'],
 ['automated pattern recognition',
  'classification',
  'text classification',
  'pattern recognition',
  'pattern classification',
  'pattern matching',
  'statistical pattern recognition',
  'object recognition'],
 ['predictive modeling',
  'predictive analytics',
  'prediction',
  'bioinformatics'],
 ['assembler'],
 ['python'],
 ['javascript'],
 ['reinforcement learning',
  'semi-supervised learning',
  'unsupervised learning',
  'advanced machine learning',
  'supervised learning',
  'statistical learning',
  'machine learning'],
 ['association rule mining',
  'svd',
  'computational intelligence',
  'high-dimensional data analysis',
  'data processing and computer science',
  'social network analysis',
  'web devops',
  'artificial intelligence',
  'feature extraction',
  'fuzzy clustering',
  'c++',
  'large scale data analysis',
  'neural networks',
  'data preparation',
  'genetic programming',
  'a